In [3]:
pip install transformers

     |████████████████████████████████| 1.3MB 9.2MB/s 
     |████████████████████████████████| 2.9MB 24.8MB/s 
     |████████████████████████████████| 1.1MB 43.4MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=21415ada7471d7a7d2acd7ebb174b52c9ed913dbd7afb99fa28ded0864322d9e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [33]:
! wget "https://zenodo.org/record/1489920/files/articles-training-byarticle-20181122.zip"
! unzip "articles-training-byarticle-20181122.zip"
# ! wget "https://zenodo.org/r000064ecord/1489920/files/articles-training-bypublisher-20181122.zip"
# ! unzip "articles-training-bypublisher-20181122.zip"
# ! wget "https://zenodo.org/record/1489920/files/articles-validation-bypublisher-20181122.zip"
# ! unzip "articles-validation-bypublisher-20181122.zip" 

--2020-11-18 17:24:04--  https://zenodo.org/record/1489920/files/articles-training-byarticle-20181122.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971841 (949K) [application/octet-stream]
Saving to: ‘articles-training-byarticle-20181122.zip’

articles-training-b 100%[===================>] 949.06K  1.00MB/s    in 0.9s    

2020-11-18 17:24:07 (1.00 MB/s) - ‘articles-training-byarticle-20181122.zip’ saved [971841/971841]

Archive:  articles-training-byarticle-20181122.zip
  inflating: articles-training-byarticle-20181122.xml  


In [34]:
import xml.sax
files = ["articles-training-byarticle-20181122.xml" ]
# files =["articles-training-bypublisher-20181122.xml"]
# files = ["articles-validation-bypublisher-20181122.zip"]
# files = ["ground-truth-training-bypublisher-20181122.xml"]

In [35]:
! wget "https://zenodo.org/record/1489920/files/ground-truth-training-byarticle-20181122.zip"
! unzip "ground-truth-training-byarticle-20181122.zip"


--2020-11-18 17:24:10--  https://zenodo.org/record/1489920/files/ground-truth-training-byarticle-20181122.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28511 (28K) [application/octet-stream]
Saving to: ‘ground-truth-training-byarticle-20181122.zip.1’

ground-truth-traini 100%[===================>]  27.84K   180KB/s    in 0.2s    

2020-11-18 17:24:11 (180 KB/s) - ‘ground-truth-training-byarticle-20181122.zip.1’ saved [28511/28511]

Archive:  ground-truth-training-byarticle-20181122.zip
replace ground-truth-training-byarticle-20181122.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [36]:
articles = {}
ground_dict ={}

In [38]:
# for articleTraining

class DataParser(xml.sax.ContentHandler):

    def __init__(self):
        self.title = ""
        self.text = ""
        self.id = ""
        self.date = ""

    def startElement(self, tag, attributes):
        if tag == "article":
            self.text = ""
            self.date = ""
            self.title = ""
            self.title = attributes["title"]
            self.id = attributes["id"]
            if "published-at" in attributes:
                self.date = attributes["published-at"]

    def characters(self, content):

        self.text += content

    def endElement(self, tag):
        if tag == "article":
            lis = [self.date, self.title, self.text]
            articles[self.id] = lis  

In [39]:
class DataParserGroundTruth(xml.sax.ContentHandler):
    def __init__(self):
        self.d = {}
        self.id = ""

    def startElement(self, tag, attributes):
        atts = ["hyperpartisan", "labeled-by", "url", "bias"]
        if tag == "article":
            self.d = {}
            self.id = ""
            self.id = attributes["id"]
            for a in atts:
                if a in attributes:
                    self.d[a] = attributes[a]
        if self.id != "":
            ground_dict[self.id] = self.d



In [40]:
parser = xml.sax.make_parser()
handler = DataParser()
parser.setContentHandler(handler)
for f in files:
      print(f)
      parser.parse(f)

# print(articles["0000007"])

articles-training-byarticle-20181122.xml


In [41]:
Parser = xml.sax.make_parser()
handler = DataParserGroundTruth()
Parser.setContentHandler(handler)
Parser.parse('ground-truth-training-byarticle-20181122.xml')

In [42]:
ground_dict['0000000']

{'hyperpartisan': 'true',
 'labeled-by': 'article',
 'url': 'https://www.opednews.com/articles/Kucinich-Reclaiming-the-m-by-Dennis-Kucinich-Banks_Debt_Funding_Money-170910-112.html'}

In [43]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score
from transformers import AutoModel, AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
tqdm.pandas()

In [44]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
dataset = []

In [47]:
keys = list(articles.keys())

In [48]:
for k in keys:
  d = {}
  d['id'] = k
  d['title'] = articles[k][1]
  d['text'] = articles[k][2]
  d['hyperpartisan'] = ground_dict[k]['hyperpartisan']
  d['url'] = ground_dict[k]['url']
  d['labeled_by'] = ground_dict[k]['labeled-by']
  dataset.append(d)

In [49]:
train_df_articles = pd.DataFrame(dataset)

In [50]:
train_df_articles.head()

,id,title,text,hyperpartisan,url,labeled_by
0,0000000,Kucinich: Reclaiming the money power,\nFrom flickr.com: Money {MID-161793} Money ( ...,true,https://www.opednews.com/articles/Kucinich-Rec...,article
1,0000001,Trump Just Woke Up & Viciously Attacked Puerto...,Donald Trump ran on many braggadocios and larg...,true,http://bipartisanreport.com/2017/10/12/trump-j...,article
2,0000002,"Liberals wailing about gun control, but what a...",\nPhoto By Justin Sullivan/Getty Images In res...,true,https://www.reviewjournal.com/opinion/letters/...,article
3,0000003,Laremy Tunsil joins NFL players in kneeling du...,After Colin Kaepernick rightly chose to kneel ...,true,https://www.redcuprebellion.com/2017/9/24/1635...,article
4,0000004,It's 1968 All Over Again,"Almost a half-century ago, in 1968, the United...",false,https://www.realclearpolitics.com/articles/201...,article


In [51]:
stop_words = set(stopwords.words('english'))
def process_text(text):
  text = text.lower()
  text = text.translate(str.maketrans('', '', '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'))
  text = re.sub('[ ]+', ' ', text)
  text = text.strip()
  # word_tokens = re.split(r'[^A-Za-z0-9]+', text)
  # filtered_text = [w for w in word_tokens if w not in stop_words]
  # text = (' ').join(filtered_text)
  return text

In [52]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [53]:
train_df_articles['processed_text'] = train_df_articles['text'].progress_apply(process_text)

In [54]:
def nsents(text):
  return len(sent_tokenize(text))

In [55]:
def nwords(text):
  return len(word_tokenize(text))

In [56]:
train_df_articles['nsents'] = train_df_articles['processed_text'].apply(nsents)
train_df_articles['nwords'] = train_df_articles['processed_text'].apply(nwords)

In [57]:
data = []
for i,j in zip(train_df_articles['nwords'], train_df_articles['nsents']):
  data.append((j,i))

In [58]:
train_df_articles.head()

,id,title,text,hyperpartisan,url,labeled_by,processed_text,nsents,nwords
0,0000000,Kucinich: Reclaiming the money power,\nFrom flickr.com: Money {MID-161793} Money ( ...,true,https://www.opednews.com/articles/Kucinich-Rec...,article,from flickr.com money mid161793 money image by...,61,1526
1,0000001,Trump Just Woke Up & Viciously Attacked Puerto...,Donald Trump ran on many braggadocios and larg...,true,http://bipartisanreport.com/2017/10/12/trump-j...,article,donald trump ran on many braggadocios and larg...,11,257
2,0000002,"Liberals wailing about gun control, but what a...",\nPhoto By Justin Sullivan/Getty Images In res...,true,https://www.reviewjournal.com/opinion/letters/...,article,photo by justin sullivangetty images in respon...,8,188
3,0000003,Laremy Tunsil joins NFL players in kneeling du...,After Colin Kaepernick rightly chose to kneel ...,true,https://www.redcuprebellion.com/2017/9/24/1635...,article,after colin kaepernick rightly chose to kneel ...,22,338
4,0000004,It's 1968 All Over Again,"Almost a half-century ago, in 1968, the United...",false,https://www.realclearpolitics.com/articles/201...,article,almost a halfcentury ago in 1968 the united st...,40,848


In [59]:
label_dict = {'true': 1, 'false': 0}

In [60]:
train_df_articles['label'] = train_df_articles['hyperpartisan'].replace(label_dict)

In [61]:
transformer_config = 'albert-base-v2'
# transformer_config = 'roberta-base'

# model_type=""
# model_type = ""
model_type = "hierarchical"

In [62]:
tokenizer = AutoTokenizer.from_pretrained(transformer_config, do_lower_case=True)

In [63]:
train_data, val_data = train_test_split(train_df_articles[['id', 'processed_text', 'label']], test_size=0.2, stratify=train_df_articles.label.values)

In [64]:
train_data.columns = ['id', 'text', 'label']
val_data.columns = ['id', 'text', 'label']

In [65]:
train_data.head()

,id,text,label
420,0000420,writing in the yorkshire post sir bernard ingh...,0
312,0000312,salah abdeslam wasnt in belgian court as verdi...,0
439,0000439,the level of doublethink involved in being a c...,1
19,0000019,the world is watching and so is oil as the dea...,1
111,0000111,nation media group reporter walter menyas arre...,0


In [66]:
max_length = 512
encoded_data_train = tokenizer.batch_encode_plus(
    train_data.text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=max_length,
    return_tensors='pt',
    truncation=True
)
encoded_data_val = tokenizer.batch_encode_plus(
    val_data.text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=max_length,
    return_tensors='pt',
    truncation=True
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_data['label'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(val_data['label'].values)

In [67]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [68]:
batch_size = 8
train_loader = DataLoader(dataset_train, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(dataset_val, shuffle=True, batch_size=batch_size)

In [69]:
class Hi_TF(nn.Module):
  def __init__(self, config, emb_size):
    super().__init__()
    self.tf1 = AutoModel.from_pretrained(config)
    self.tf2 = AutoModelForSequenceClassification.from_pretrained(
        config, num_labels=2, output_attentions=False, output_hidden_states=False)
    self.emb_size = emb_size
    self.end_token = tokenizer.convert_tokens_to_ids('.')

  def forward(self, inputs):
    inputs1 = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    }
    outputs = self.tf1(**inputs1)
    hs = outputs[0]
    print(hs.shape)
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']
    batch_size = input_ids.shape[0]
    final_embeds = torch.zeros(1, 100, self.emb_size)
    final_amasks = torch.zeros(1, 100)
    for b in range(batch_size):
      idx = []
      for i, l in enumerate(input_ids[b]):
        if l == self.end_token:
          idx.append(i)
      if len(idx) > 0:
        embeds = torch.stack([hs[b][i] for i in idx])
      else:
        embeds = hs[b][i-1]
        embeds = embeds.unsqueeze(0)
      k = embeds.shape[0]
      amasks = torch.ones(k)
      zeros = torch.zeros(max(1, 100-k), self.emb_size)
      embeds = torch.cat((embeds.to(device), zeros.to(device)), dim=0)
      amasks = torch.cat((amasks.to(device), torch.zeros(max(1, 100-k)).to(device)), dim=0)
      if b==0:
        final_embeds = embeds.unsqueeze(0).to(device)
        final_amasks = amasks.unsqueeze(0).to(device)
      else:
        final_embeds = torch.cat((final_embeds.to(device), embeds.unsqueeze(0).to(device)), dim=0)
        final_amasks = torch.cat((final_amasks.to(device), amasks.unsqueeze(0).to(device)), dim=0)
    inputs2 = {
        'inputs_embeds': final_embeds,
        'attention_mask': final_amasks,
        'labels': inputs['labels']
    }
    outputs2 = self.tf2(**inputs2)
    return outputs2

In [80]:
if model_type == "hierarchical":
  model = Hi_TF(transformer_config, 768)
else:
  model = AutoModelForSequenceClassification.from_pretrained(transformer_config, num_labels=len(label_dict), output_hidden_states=False, output_attentions=False)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [82]:
lr = 5e-6
if model_type == "hierarchical":
  epochs = 10
else:
  epochs = 10
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=len(train_loader)*epochs)

In [83]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    log_loss = nn.CrossEntropyLoss()
    # coef, p = spearmanr(scores_flat, preds_flat)
    # return log_loss(torch.tensor(preds).float(), torch.tensor(labels)), f1_score(labels_flat, preds_flat, average='binary', pos_label=0), fbeta_score(labels_flat, preds_flat, average='binary', beta=0.5, pos_label=0)
    return log_loss(torch.tensor(preds).float(), torch.tensor(labels)), f1_score(labels_flat, preds_flat, average='weighted')

In [84]:
def accuracy_per_class(preds, labels):
    # label_dict_inverse = {v: k for k, v in label_dict.items()}
    label_dict_inverse = {1: 'Hyperpartisan', 0: 'Not hyperpartisan'}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    l_preds=0
    l_true=0
    accuracies = []
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        l_preds += len(y_preds[y_preds==label])
        l_true += len(y_true)
        print(f'Class: {label_dict_inverse[label]}')
        accuracies.append(f'{len(y_preds[y_preds==label])}/{len(y_true)}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} \t {len(y_preds[y_preds==label])/len(y_true)}\n')
    print(f'Total Accuracy: {l_preds/l_true}')
    return accuracies, l_preds/l_true

In [85]:
import random

seed_val = 17
random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed(seed_val)

In [86]:
def evaluate(dataloader_val):

    model.eval()
    # model2.eval()
    loss_val_total = 0
    predictions, true_vals, scores = [], [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                  # 'scores':         batch[3]
                 }

        with torch.no_grad():
            if model_type == "hierarchical":
              outputs = model(inputs)
            else:
              outputs = model(**inputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    return loss_val_avg, predictions, true_vals

In [87]:
best_loss = float('inf')
my_dict = []
for epoch in tqdm(range(1+10, epochs+1+10)):
    
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(train_loader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels':         batch[2].to(device),
                 }
        if model_type == "hierarchical":
          outputs = model(inputs)
        else:
          outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_loader)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_loader)
    torch.save(model.state_dict(), f'finetuned_{model_type+transformer_config}_epoch_{epoch}.model')
    celoss, val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'Cross Entropy Loss: {celoss}')
    torch.cuda.empty_cache()
    accuracies, total_acc = accuracy_per_class(predictions, true_vals)
    my_dict.append({'epoch': epoch, 'train_loss': loss_train_avg, 'loss': val_loss, 'f1': val_f1, 'Not Hyperpartisan': accuracies[0], 'Hyperpartisan': accuracies[1], 'Total Accuracy': total_acc})

torch.Size([8, 512, 768])


RuntimeError: ignored

In [ ]:
results = pd.DataFrame(my_dict)
results.to_csv(f'{model_type}{transformer_config}_{max_length}_{lr}_{batch_size}.csv', index=False)

In [ ]:
!cp *.csv /content/drive/MyDrive/